In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import random

import pandas as pd

In [ ]:
import os

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# Cài Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install -y ./google-chrome-stable_current_amd64.deb

--2025-05-04 01:51:49--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 64.233.181.136, 64.233.181.93, 64.233.181.91, ...
Connecting to dl.google.com (dl.google.com)|64.233.181.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116169896 (111M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.2’

google-chrome-stabl 100%[===================>] 110.79M   386MB/s    in 0.3s    

2025-05-04 01:51:50 (386 MB/s) - ‘google-chrome-stable_current_amd64.deb.2’ saved [116169896/116169896]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
google-chrome-stable is already the newest version (136.0.7103.59-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!google-chrome --version

Google Chrome 136.0.7103.59 


In [ ]:
!wget https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.39/linux64/chromedriver-linux64.zip
!unzip chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

--2025-05-04 01:48:02--  https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.39/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.207, 74.125.132.207, 74.125.201.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9593043 (9.1M) [application/zip]
Saving to: ‘chromedriver-linux64.zip.1’

chromedriver-linux6 100%[===================>]   9.15M  --.-KB/s    in 0.1s    

2025-05-04 01:48:02 (87.1 MB/s) - ‘chromedriver-linux64.zip.1’ saved [9593043/9593043]

Archive:  chromedriver-linux64.zip
replace chromedriver-linux64/LICENSE.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: chromedriver-linux64/LICENSE.chromedriver  
replace chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflati

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [ ]:
from datetime import datetime

In [ ]:
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.binary_location = "/usr/bin/google-chrome"

In [ ]:
service = Service("/usr/bin/chromedriver")
browser = webdriver.Chrome(service=service, options=options)

In [ ]:
def craw_stock_news(ticker, start_date, end_date, roof_path):
  df = pd.DataFrame(data = None, columns = ['ticker','time', 'tittle', 'url'])
  browser.get("https://cafef.vn/du-lieu/tin-doanh-nghiep/"+str(ticker)+"/event.chn")
  print(browser.title)
  flag = True
  while (flag):
    li_elements = browser.find_elements(By.XPATH, "//div[@id='divEvents']/ul/li")
    for idx, li in enumerate(li_elements, start= 1):
      span = li.find_element(By.TAG_NAME, "span")
      curent_date_of_new = span.text
      curent_date_of_new_convert = datetime.strptime(curent_date_of_new, "%d/%m/%Y %H:%M")
      if (curent_date_of_new_convert < start_date):
        print("Đã hết thời gian cần tìm")
        flag = False
        break
      if curent_date_of_new_convert >= start_date and curent_date_of_new_convert <= end_date:
        time = curent_date_of_new
        url = li.find_element(By.TAG_NAME, "a").get_attribute('href')
        title = li.find_element(By.TAG_NAME, "a").text
        print('Đang lấy bài báo với thông tin như sau: ')
        print(time)
        print(title)
        print(url)
        new_row = {'ticker': ticker, 'time':time, 'tittle': title, 'url':url}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
      if (idx == len(li_elements)):
        print("Đây là phần tử cuối cùng")
        # Click vào nút xem tiếp
        try:
          btn_next_element = browser.find_element(By.XPATH, "//span[@id='spanNext']/a")
          browser.execute_script("arguments[0].click();", btn_next_element)
          sleep(random.uniform(1, 2))
          print("Cuyển qua trang tiếp theo")
        except Exception as e:
          print(f"Không thể click nút tiếp: {e}")
          flag = False
          break
  # Tạo thư mục nếu chưa tồn tại
  os.makedirs(roof_path, exist_ok=True)
  file_path = roof_path + "/" + str(ticker) + ".xlsx"
  with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)
  print("Đã lưu file excel cho cổ phiếu " + str(ticker))
  # 5. Đóng browser
  # browser.close()

In [ ]:
end_date = datetime.strptime('31/12/2024 00:00', "%d/%m/%Y %H:%M")
start_date = datetime.strptime('01/01/2024 00:00', "%d/%m/%Y %H:%M")

In [ ]:
craw_stock_news("ACB", start_date, end_date, roof_path="data_test")

ACB : Tin tức doanh nghiệp niêm yết | CafeF.vn
Đang lấy bài báo với thông tin như sau: 
31/12/2024 00:00
ACB: Ông Nguyễn Văn Hòa thôi kiêm nhiệm chức Giám đốc Tài chính từ 1.1.2025
https://cafef.vn/du-lieu/ACB-2109424/acb-ong-nguyen-van-hoa-thoi-kiem-nhiem-chuc-giam-doc-tai-chinh-tu-112025.chn
Đang lấy bài báo với thông tin như sau: 
01/11/2024 07:37
Những cái tên lãi lớn nhất sàn chứng khoán quý 3/2024: Vinhomes 'đòi' lại ngôi vương, một cái tên gây bất ngờ
https://cafef.vn/nhung-cai-ten-lai-lon-nhat-san-chung-khoan-quy-32024-vinhomes-doi-lai-ngoi-vuong-mot-cai-ten-gay-bat-ngo-188241101073751618.chn
Đang lấy bài báo với thông tin như sau: 
21/10/2024 00:00
ACB: Nghị quyết HĐQT số 4299/TCQĐ-HĐQT.24, 4303/TCQĐ-HĐQT.24, 4304/TCQĐ-HĐQT.24 ngày 21/10/2024
https://cafef.vn/du-lieu/ACB-2072549/acb-nghi-quyet-hdqt-so-4299tcqdhdqt24-4303tcqdhdqt24-4304tcqdhdqt24-ngay-21102024.chn
Đang lấy bài báo với thông tin như sau: 
27/09/2024 00:00
ACB: Nhắc nhở chậm CBTT Nghị quyết HĐQT số 910/TCQĐ-HĐQT.

In [ ]:
def craw_stock_news_for_vn30(tickers, start_date, end_date, roof_path):
  for ticker in tickers:
    try:
      craw_stock_news(ticker, start_date, end_date, roof_path)
    except Exception as e:
      print(f"Lỗi khi xử lý cổ phiếu {ticker}: {e}")
      continue

In [ ]:
# tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
tickers = ['LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
end_date = datetime.strptime('31/12/2023 00:00', "%d/%m/%Y %H:%M")
start_date = datetime.strptime('01/01/2019 00:00', "%d/%m/%Y %H:%M")

In [ ]:
# crawl new for train
craw_stock_news_for_vn30(tickers, start_date, end_date, roof_path="/content/drive/MyDrive/thacsi/LuanVan/PredictStock/SA/v1/data/url_title_new_vn30/train_data")

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
https://cafef.vn/du-lieu/vpb-1811746/vpbank-va-amazon-web-services-to-chuc-hoi-thao-chuyen-doi-so-ngan-hang.chn
Đang lấy bài báo với thông tin như sau: 
21/09/2023 00:00
VPB: Giao dịch của người nội bộ tham gia ESOP năm 2023
https://cafef.vn/du-lieu/vpb-1811861/vpb-giao-dich-cua-nguoi-noi-bo-tham-gia-esop-nam-2023.chn
Đang lấy bài báo với thông tin như sau: 
18/09/2023 00:00
VPB: Sử dụng cổ phiếu quỹ để bán cho người lao động
https://cafef.vn/du-lieu/vpb-1808750/vpb-su-dung-co-phieu-quy-de-ban-cho-nguoi-lao-dong.chn
Đang lấy bài báo với thông tin như sau: 
18/09/2023 00:00
VPB: CBTT về việc UBCK nhận đủ hồ sơ đăng ký chào bán cổ phiếu riêng lẻ
https://cafef.vn/du-lieu/vpb-1808753/vpb-cbtt-ve-viec-ubck-nhan-du-ho-so-dang-ky-chao-ban-co-phieu-rieng-le.chn
Đang lấy bài báo với thông tin như sau: 
15/09/2023 13:30
Điểm chạm hạnh phúc của VPBanker
https://cafef.vn/du-lieu/vpb-1804153/diem-cham-hanh-phuc-cua-vpbanker.chn
Đang lấy bài b

In [ ]:
# crawl new for test
tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
# tickers = ['LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
end_date = datetime.strptime('31/12/2024 00:00', "%d/%m/%Y %H:%M")
start_date = datetime.strptime('01/01/2024 00:00', "%d/%m/%Y %H:%M")
craw_stock_news_for_vn30(tickers, start_date, end_date, roof_path="/content/drive/MyDrive/thacsi/LuanVan/PredictStock/SA/v1/data/url_title_new_vn30/test_data")

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
https://cafef.vn/du-lieu/ssi-2088506/ssi-da-phat-hanh-150913867-co-phieu-ra-cong-chung.chn
Đang lấy bài báo với thông tin như sau: 
22/11/2024 00:00
SSI: Bà Nguyễn Thị Thanh Hà - Giám đốc Tài chính đã mua 300.000 cp
https://cafef.vn/du-lieu/ssi-2088509/ssi-ba-nguyen-thi-thanh-ha-giam-doc-tai-chinh-da-mua-300000-cp.chn
Đang lấy bài báo với thông tin như sau: 
22/11/2024 00:00
SSI: Ông Nguyễn Đức Thông - Phó Tổng giám đốc đã mua 300.000 cp
https://cafef.vn/du-lieu/ssi-2088510/ssi-ong-nguyen-duc-thong-pho-tong-giam-doc-da-mua-300000-cp.chn
Đang lấy bài báo với thông tin như sau: 
19/11/2024 00:00
SSI: Giấy chứng nhận đăng ký doanh nghiệp thay đổi lần thứ 32
https://cafef.vn/du-lieu/ssi-2086890/ssi-giay-chung-nhan-dang-ky-doanh-nghiep-thay-doi-lan-thu-32.chn
Đang lấy bài báo với thông tin như sau: 
18/11/2024 00:00
SSI: Nghị quyết HĐQT thông qua kết quả thực hiện quyền mua được chào bán cổ phiếu cho CĐHH và tiếp tục phân phối CP
http